# FDE Technical Screen

## Objective
- Write a ***function*** to ***sort*** packages into stacks ***by volume*** and ***mass***.

## Rules
- A ***package*** is
  - a rectangular prism with volume defined by $V=xyz$.
  - roughly constant density with mass defined by $m=\rho V$.
  - ***bulky*** $(\beta)$ if
    - $V\geq10^6\mathrm{cm}^3$ ***or***
    - $\max(x,y,z)\geq150\mathrm{cm}$.
  - ***heavy*** $(\eta)$ if
    - $m\geq20\mathrm{kg}$.
- A stack is
  - ***STANDARD*** if $\neg\beta\land\neg\eta = \neg(\beta\lor\eta)$
  - ***SPECIAL*** if $\beta\lor\eta$
  - ***REJECTED*** if $\beta\land\eta$

In [1]:
from decimal import Decimal


def sort(width: float, height: float, length: float, mass: float) -> str:
    """Sort a package into a stack based on its dimensions and mass.

    Possible package stacks are STANDARD, SPECIAL, or REJECTED. Note that we use the
    `Decimal` class for high-precision calculation.

    Args:
        width (float): Width of the package in centimeters.
        height (float): Height of the package in centimeters.
        length (float): Length of the package in centimeters.
        mass (float): Mass of the package in kilograms.

    Returns:
        str: The uppercase string representing the appropriate stack for the given package.
    """
    if any(v <= 0 for v in [width, height, length, mass]):  # Not physically possible.
        return "REJECTED"
    width, height, length, mass = map(Decimal, [width, height, length, mass])
    is_bulky = (
        width * length * height >= 10**6 or max([length, width, height]) >= 150
    )
    is_heavy = mass >= 20
    # print(f"B:{is_bulky}, H:{is_heavy}")
    if is_bulky and is_heavy:
        return "REJECTED"
    if is_bulky or is_heavy:
        return "SPECIAL"
    return "STANDARD"

In [2]:
# Quick sanity check
sort(149.9999, 149.99999999, 149.999999, 19.9999)  # Note that V exceeds 1M!

'SPECIAL'

In [3]:
import unittest
from fractions import Fraction
from math import sqrt


class TestSortFunction(unittest.TestCase):
    def test_standard(self):
        self.assertEqual(sort(30, 20, 10, 2), "STANDARD")  # Shoe box
        self.assertEqual(sort(50, 50, 50, 10), "STANDARD")  # Computer box

    def test_bulky(self):
        self.assertEqual(sort(200, 50, 50, 10), "SPECIAL")  # max(x,y,z) > 150
        self.assertEqual(sort(50, 200, 50, 10), "SPECIAL")
        self.assertEqual(sort(50, 50, 200, 10), "SPECIAL")
        self.assertEqual(sort(50, 200, 200, 10), "SPECIAL")  # Two exceed limit
        self.assertEqual(sort(200, 50, 200, 10), "SPECIAL")
        self.assertEqual(sort(200, 200, 50, 10), "SPECIAL")
        self.assertEqual(sort(200, 200, 200, 10), "SPECIAL")  # Three exceed limit

        self.assertEqual(sort(150, 50, 50, 10), "SPECIAL")  # max(x,y,z) = 150
        self.assertEqual(sort(50, 150, 50, 10), "SPECIAL")
        self.assertEqual(sort(50, 50, 150, 10), "SPECIAL")
        self.assertEqual(sort(50, 150, 150, 10), "SPECIAL")  # Two at limit
        self.assertEqual(sort(150, 50, 150, 10), "SPECIAL")
        self.assertEqual(sort(150, 150, 50, 10), "SPECIAL")
        self.assertEqual(sort(150, 150, 150, 10), "SPECIAL")  # Three at limit

        self.assertEqual(sort(100, 100, 101, 10), "SPECIAL")  # V > 10^6
        self.assertEqual(sort(100, 100, 100, 10), "SPECIAL")  # V = 10^6

    def test_heavy(self):
        self.assertEqual(sort(50, 50, 50, 25), "SPECIAL")  # m > 20 kg
        self.assertEqual(sort(50, 50, 50, 20), "SPECIAL")  # m = 20 kg
        self.assertNotEqual(sort(50, 50, 50, 19), "SPECIAL")  # m < 20 kg
        self.assertNotEqual(sort(50, 50, 50, -20), "SPECIAL")  # invalid

    def test_rejected(self):
        self.assertEqual(sort(200, 200, 200, 30), "REJECTED")  # All dims + mass
        self.assertEqual(sort(200, 10, 10, 30), "REJECTED")  # One dim + mass
        self.assertEqual(sort(10, 200, 10, 30), "REJECTED")
        self.assertEqual(sort(10, 10, 200, 30), "REJECTED")
        self.assertEqual(sort(-1, 1, 1, 1), "REJECTED")  # Invalid input
        # Can do all combos of negative and 0 later with some generator...
        self.assertEqual(sort(-1, -1, 1, 0), "REJECTED")
        self.assertEqual(sort(-1, -1, -1, 0), "REJECTED")
        self.assertEqual(sort(-1, -1, -1, -1), "REJECTED")
        self.assertEqual(sort(1, -1, 1, 1), "REJECTED")
        self.assertEqual(sort(1, -1, -1, 1), "REJECTED")
        self.assertEqual(sort(0, -1, -1, -1), "REJECTED")
        self.assertEqual(sort(1, 0, -1, 1), "REJECTED")
        self.assertEqual(sort(1, 1, 1, -1), "REJECTED")

    def test_edge_cases(self):
        # 149.999 * y * z = 999_999.999
        const = sqrt(999_999.999 / 149.999)
        self.assertEqual(sort(149.999, const, const, 19.999), "STANDARD")  # All below
        self.assertEqual(sort(149.999, const, const, 20), "SPECIAL")  # Mass at limit
        # All dims and mass below, but volume equals limit
        const = sqrt(Fraction(1_000_000 / 149.999))
        self.assertEqual(sort(149.999, const, const, 19.999), "SPECIAL")
        self.assertEqual(sort(150, 1, 1, 19.999), "SPECIAL")  # One dim at limit

unittest.TextTestRunner().run(
    unittest.TestLoader().loadTestsFromTestCase(TestSortFunction)
)


.....
----------------------------------------------------------------------
Ran 5 tests in 0.003s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>